# Week 18: Portfolio Optimization Theory

## Comprehensive Guide to Modern Portfolio Construction

This notebook covers the theoretical foundations and practical implementations of:

1. **Markowitz Mean-Variance Optimization** - The foundational framework
2. **Black-Litterman Model** - Bayesian approach incorporating investor views
3. **Risk Parity** - Equal risk contribution portfolios
4. **Hierarchical Risk Parity (HRP)** - Machine learning-based allocation

---

### Key Concepts Covered:
- Mathematical formulations and optimization objectives
- Advantages and limitations of each approach
- Practical implementation considerations
- Comparison across methods

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import squareform
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Libraries imported successfully!")

## 2. Sample Data Generation

We'll create synthetic asset return data to demonstrate all optimization methods.

In [ ]:
# Define assets
assets = ['US_Equity', 'Intl_Equity', 'EM_Equity', 'US_Bonds', 'Corp_Bonds', 
          'Commodities', 'REITs', 'Gold']
n_assets = len(assets)

# Expected annual returns (realistic estimates)
expected_returns = np.array([0.08, 0.07, 0.10, 0.03, 0.04, 0.05, 0.07, 0.02])

# Annual volatilities
volatilities = np.array([0.16, 0.18, 0.25, 0.05, 0.07, 0.20, 0.19, 0.15])

# Correlation matrix (realistic cross-asset correlations)
correlation_matrix = np.array([
    [1.00, 0.85, 0.75, 0.10, 0.30, 0.40, 0.65, 0.05],   # US Equity
    [0.85, 1.00, 0.80, 0.05, 0.25, 0.45, 0.60, 0.10],   # Intl Equity
    [0.75, 0.80, 1.00, 0.00, 0.20, 0.50, 0.55, 0.15],   # EM Equity
    [0.10, 0.05, 0.00, 1.00, 0.70, -0.10, 0.20, 0.30],  # US Bonds
    [0.30, 0.25, 0.20, 0.70, 1.00, 0.10, 0.35, 0.20],   # Corp Bonds
    [0.40, 0.45, 0.50, -0.10, 0.10, 1.00, 0.40, 0.35],  # Commodities
    [0.65, 0.60, 0.55, 0.20, 0.35, 0.40, 1.00, 0.15],   # REITs
    [0.05, 0.10, 0.15, 0.30, 0.20, 0.35, 0.15, 1.00],   # Gold
])

# Covariance matrix
cov_matrix = np.outer(volatilities, volatilities) * correlation_matrix

# Display summary
summary_df = pd.DataFrame({
    'Asset': assets,
    'Expected Return': expected_returns,
    'Volatility': volatilities
})
print("Asset Summary:")
print(summary_df.to_string(index=False))
print(f"\nCovariance Matrix Shape: {cov_matrix.shape}")

---

## 3. Markowitz Mean-Variance Optimization (MVO)

### 3.1 Theory and Mathematical Foundation

**Harry Markowitz (1952)** introduced Modern Portfolio Theory, earning the Nobel Prize in Economics (1990).

### Key Insight
Investors should not look at individual asset risk/return, but at how assets behave **together** in a portfolio.

### Mathematical Formulation

**Portfolio Return:**
$$\mu_p = \sum_{i=1}^{n} w_i \mu_i = \mathbf{w}^T \boldsymbol{\mu}$$

**Portfolio Variance:**
$$\sigma_p^2 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij} = \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

### Optimization Problem

**Minimize variance for a given target return:**

$$\min_{\mathbf{w}} \quad \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

Subject to:
- $\mathbf{w}^T \boldsymbol{\mu} = \mu_{target}$ (target return constraint)
- $\mathbf{w}^T \mathbf{1} = 1$ (fully invested)
- $w_i \geq 0$ (no short selling, optional)

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Efficient Frontier** | Set of portfolios with maximum return for each risk level |
| **Minimum Variance Portfolio (MVP)** | Portfolio with lowest possible volatility |
| **Tangency Portfolio** | Portfolio with highest Sharpe ratio |
| **Capital Market Line (CML)** | Line from risk-free rate through tangency portfolio |

### 3.2 Implementation: Markowitz Optimization

In [ ]:
class MarkowitzOptimizer:
    """
    Mean-Variance Portfolio Optimizer using Markowitz framework.
    """
    
    def __init__(self, expected_returns, cov_matrix, asset_names=None):
        """
        Initialize optimizer with expected returns and covariance matrix.
        
        Parameters:
        -----------
        expected_returns : array-like
            Expected returns for each asset
        cov_matrix : array-like
            Covariance matrix of asset returns
        asset_names : list, optional
            Names of assets
        """
        self.mu = np.array(expected_returns)
        self.sigma = np.array(cov_matrix)
        self.n_assets = len(expected_returns)
        self.asset_names = asset_names if asset_names else [f'Asset_{i}' for i in range(self.n_assets)]
        
    def portfolio_return(self, weights):
        """Calculate portfolio expected return."""
        return np.dot(weights, self.mu)
    
    def portfolio_volatility(self, weights):
        """Calculate portfolio volatility (standard deviation)."""
        return np.sqrt(np.dot(weights.T, np.dot(self.sigma, weights)))
    
    def portfolio_sharpe(self, weights, rf=0.02):
        """Calculate portfolio Sharpe ratio."""
        return (self.portfolio_return(weights) - rf) / self.portfolio_volatility(weights)
    
    def minimize_variance(self, target_return=None, allow_short=False):
        """
        Find minimum variance portfolio, optionally for a target return.
        
        Parameters:
        -----------
        target_return : float, optional
            Target portfolio return constraint
        allow_short : bool
            Whether to allow short selling
        
        Returns:
        --------
        dict : Optimization results including weights, return, volatility
        """
        # Initial guess: equal weights
        init_weights = np.ones(self.n_assets) / self.n_assets
        
        # Constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Weights sum to 1
        
        if target_return is not None:
            constraints.append({
                'type': 'eq', 
                'fun': lambda w: self.portfolio_return(w) - target_return
            })
        
        # Bounds
        bounds = None if allow_short else tuple((0, 1) for _ in range(self.n_assets))
        
        # Optimize
        result = minimize(
            lambda w: self.portfolio_volatility(w),
            init_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        
        if not result.success:
            print(f"Warning: Optimization did not converge - {result.message}")
        
        weights = result.x
        return {
            'weights': weights,
            'return': self.portfolio_return(weights),
            'volatility': self.portfolio_volatility(weights),
            'sharpe': self.portfolio_sharpe(weights)
        }
    
    def maximize_sharpe(self, rf=0.02, allow_short=False):
        """
        Find the tangency portfolio (maximum Sharpe ratio).
        
        Parameters:
        -----------
        rf : float
            Risk-free rate
        allow_short : bool
            Whether to allow short selling
        
        Returns:
        --------
        dict : Optimization results
        """
        init_weights = np.ones(self.n_assets) / self.n_assets
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = None if allow_short else tuple((0, 1) for _ in range(self.n_assets))
        
        # Minimize negative Sharpe ratio
        result = minimize(
            lambda w: -self.portfolio_sharpe(w, rf),
            init_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        
        weights = result.x
        return {
            'weights': weights,
            'return': self.portfolio_return(weights),
            'volatility': self.portfolio_volatility(weights),
            'sharpe': self.portfolio_sharpe(weights, rf)
        }
    
    def efficient_frontier(self, n_points=100, allow_short=False):
        """
        Generate the efficient frontier.
        
        Parameters:
        -----------
        n_points : int
            Number of points on the frontier
        allow_short : bool
            Whether to allow short selling
        
        Returns:
        --------
        tuple : Arrays of returns, volatilities, and weights
        """
        # Find minimum and maximum achievable returns
        min_ret = self.minimize_variance(allow_short=allow_short)['return']
        max_ret = max(self.mu) if not allow_short else max(self.mu) * 1.5
        
        target_returns = np.linspace(min_ret, max_ret, n_points)
        frontier_volatilities = []
        frontier_returns = []
        frontier_weights = []
        
        for target in target_returns:
            try:
                result = self.minimize_variance(target_return=target, allow_short=allow_short)
                frontier_returns.append(result['return'])
                frontier_volatilities.append(result['volatility'])
                frontier_weights.append(result['weights'])
            except:
                continue
        
        return np.array(frontier_returns), np.array(frontier_volatilities), np.array(frontier_weights)

# Initialize optimizer
mvo = MarkowitzOptimizer(expected_returns, cov_matrix, assets)
print("Markowitz Optimizer initialized successfully!")

### 3.3 Key Portfolios: Minimum Variance and Maximum Sharpe

In [ ]:
# Find key portfolios
min_var_portfolio = mvo.minimize_variance(allow_short=False)
max_sharpe_portfolio = mvo.maximize_sharpe(rf=0.02, allow_short=False)

print("=" * 60)
print("MINIMUM VARIANCE PORTFOLIO")
print("=" * 60)
print(f"Expected Return: {min_var_portfolio['return']:.2%}")
print(f"Volatility:      {min_var_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio:    {min_var_portfolio['sharpe']:.3f}")
print("\nWeights:")
for asset, weight in zip(assets, min_var_portfolio['weights']):
    if weight > 0.001:
        print(f"  {asset:15s}: {weight:.2%}")

print("\n" + "=" * 60)
print("MAXIMUM SHARPE PORTFOLIO (Tangency Portfolio)")
print("=" * 60)
print(f"Expected Return: {max_sharpe_portfolio['return']:.2%}")
print(f"Volatility:      {max_sharpe_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio:    {max_sharpe_portfolio['sharpe']:.3f}")
print("\nWeights:")
for asset, weight in zip(assets, max_sharpe_portfolio['weights']):
    if weight > 0.001:
        print(f"  {asset:15s}: {weight:.2%}")

### 3.4 Efficient Frontier Visualization

In [ ]:
# Generate efficient frontier
ef_returns, ef_volatilities, ef_weights = mvo.efficient_frontier(n_points=100, allow_short=False)

# Generate random portfolios for comparison
n_random = 5000
random_weights = np.random.dirichlet(np.ones(n_assets), n_random)
random_returns = np.array([mvo.portfolio_return(w) for w in random_weights])
random_volatilities = np.array([mvo.portfolio_volatility(w) for w in random_weights])
random_sharpes = (random_returns - 0.02) / random_volatilities

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

# Random portfolios
scatter = ax.scatter(random_volatilities, random_returns, c=random_sharpes, 
                     cmap='viridis', alpha=0.3, s=10, label='Random Portfolios')
plt.colorbar(scatter, ax=ax, label='Sharpe Ratio')

# Efficient frontier
ax.plot(ef_volatilities, ef_returns, 'r-', linewidth=3, label='Efficient Frontier')

# Key portfolios
ax.scatter(min_var_portfolio['volatility'], min_var_portfolio['return'], 
           c='blue', marker='*', s=400, zorder=5, edgecolors='white', 
           label=f"Min Variance (SR: {min_var_portfolio['sharpe']:.2f})")
ax.scatter(max_sharpe_portfolio['volatility'], max_sharpe_portfolio['return'], 
           c='gold', marker='*', s=400, zorder=5, edgecolors='black',
           label=f"Max Sharpe (SR: {max_sharpe_portfolio['sharpe']:.2f})")

# Individual assets
ax.scatter(volatilities, expected_returns, c='red', marker='D', s=100, 
           zorder=5, edgecolors='black', label='Individual Assets')
for i, asset in enumerate(assets):
    ax.annotate(asset, (volatilities[i], expected_returns[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# Capital Market Line
rf = 0.02
cml_x = np.linspace(0, max(ef_volatilities) * 1.2, 100)
cml_y = rf + max_sharpe_portfolio['sharpe'] * cml_x
ax.plot(cml_x, cml_y, 'g--', linewidth=2, label='Capital Market Line')

ax.set_xlabel('Volatility (Standard Deviation)', fontsize=12)
ax.set_ylabel('Expected Return', fontsize=12)
ax.set_title('Markowitz Efficient Frontier', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=9)
ax.set_xlim(0, max(volatilities) * 1.3)
ax.set_ylim(0, max(expected_returns) * 1.3)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.5 Limitations of Mean-Variance Optimization

| Limitation | Description | Impact |
|------------|-------------|--------|
| **Estimation Error** | Small changes in inputs → large changes in weights | Unstable portfolios |
| **Concentration Risk** | Often produces extreme positions | Poor diversification |
| **Input Sensitivity** | Expected returns are notoriously hard to estimate | Garbage in, garbage out |
| **Single Period** | Ignores transaction costs, rebalancing | Impractical for implementation |
| **Assumes Normal Returns** | Real returns have fat tails, skewness | Underestimates tail risk |
| **No Views Incorporated** | Only uses historical data | Ignores forward-looking information |

> **"Mean-variance optimization is an error-maximization machine."** - Richard Michaud

---

## 4. Black-Litterman Model

### 4.1 Theory and Mathematical Foundation

**Black & Litterman (1992)** developed a Bayesian approach to address Markowitz limitations.

### Key Innovation
Combines **market equilibrium returns** with **investor views** to produce more stable, intuitive portfolios.

### The Framework

**Step 1: Start with Market Equilibrium (Prior)**

The implied equilibrium excess returns are derived from market capitalization weights:

$$\boldsymbol{\Pi} = \delta \boldsymbol{\Sigma} \mathbf{w}_{mkt}$$

Where:
- $\boldsymbol{\Pi}$ = Vector of implied equilibrium excess returns
- $\delta$ = Risk aversion coefficient (typically 2.5-3.5)
- $\boldsymbol{\Sigma}$ = Covariance matrix
- $\mathbf{w}_{mkt}$ = Market capitalization weights

**Step 2: Express Investor Views**

Views are expressed as:
$$\mathbf{P} \cdot \mathbf{\mu} = \mathbf{Q} + \boldsymbol{\epsilon}$$

Where:
- $\mathbf{P}$ = Pick matrix (which assets are involved in each view)
- $\mathbf{Q}$ = View expected returns
- $\boldsymbol{\Omega}$ = Uncertainty in views (diagonal covariance matrix)

**Step 3: Combine Prior and Views (Posterior)**

$$E[\mathbf{R}] = [(\tau \boldsymbol{\Sigma})^{-1} + \mathbf{P}^T \boldsymbol{\Omega}^{-1} \mathbf{P}]^{-1} [(\tau \boldsymbol{\Sigma})^{-1} \boldsymbol{\Pi} + \mathbf{P}^T \boldsymbol{\Omega}^{-1} \mathbf{Q}]$$

Where $\tau$ is a scalar indicating uncertainty in the prior (typically 0.025-0.05).

### Types of Views

| View Type | Description | Example |
|-----------|-------------|---------|
| **Absolute** | Single asset will return X% | "US Equity will return 10%" |
| **Relative** | Asset A will outperform B by X% | "US Equity will beat EM by 2%" |

### 4.2 Implementation: Black-Litterman Model

In [ ]:
class BlackLittermanOptimizer:
    """
    Black-Litterman Portfolio Optimizer.
    Combines market equilibrium with investor views using Bayesian framework.
    """
    
    def __init__(self, cov_matrix, market_weights, risk_aversion=2.5, tau=0.05, 
                 risk_free_rate=0.02, asset_names=None):
        """
        Initialize Black-Litterman optimizer.
        
        Parameters:
        -----------
        cov_matrix : array-like
            Covariance matrix of asset returns
        market_weights : array-like
            Market capitalization weights
        risk_aversion : float
            Market risk aversion coefficient (delta)
        tau : float
            Scaling factor for prior uncertainty
        risk_free_rate : float
            Risk-free rate
        asset_names : list, optional
            Names of assets
        """
        self.sigma = np.array(cov_matrix)
        self.w_mkt = np.array(market_weights)
        self.delta = risk_aversion
        self.tau = tau
        self.rf = risk_free_rate
        self.n_assets = len(market_weights)
        self.asset_names = asset_names if asset_names else [f'Asset_{i}' for i in range(self.n_assets)]
        
        # Calculate implied equilibrium returns (prior)
        self.pi = self.implied_equilibrium_returns()
        
    def implied_equilibrium_returns(self):
        """
        Calculate implied equilibrium excess returns from market weights.
        
        Pi = delta * Sigma * w_mkt
        
        Returns:
        --------
        array : Implied equilibrium excess returns
        """
        return self.delta * np.dot(self.sigma, self.w_mkt)
    
    def posterior_returns(self, P, Q, omega=None):
        """
        Calculate posterior expected returns given views.
        
        E[R] = [(tau*Sigma)^-1 + P'*Omega^-1*P]^-1 * [(tau*Sigma)^-1*Pi + P'*Omega^-1*Q]
        
        Parameters:
        -----------
        P : array-like
            Pick matrix (K x N), where K is number of views
        Q : array-like
            View expected returns (K x 1)
        omega : array-like, optional
            View uncertainty covariance matrix (K x K)
            If None, uses proportional confidence: tau * P * Sigma * P'
        
        Returns:
        --------
        array : Posterior expected returns
        """
        P = np.array(P)
        Q = np.array(Q).flatten()
        
        # If omega not provided, use He-Litterman (1999) formula
        if omega is None:
            omega = self.tau * np.dot(np.dot(P, self.sigma), P.T)
            # Make sure omega is 2D
            if omega.ndim == 0:
                omega = np.array([[omega]])
            elif omega.ndim == 1:
                omega = np.diag(omega)
        
        omega = np.atleast_2d(omega)
        
        # Prior precision and view precision
        tau_sigma_inv = np.linalg.inv(self.tau * self.sigma)
        omega_inv = np.linalg.inv(omega)
        
        # Posterior precision (information) matrix
        posterior_precision = tau_sigma_inv + np.dot(np.dot(P.T, omega_inv), P)
        
        # Posterior mean
        posterior_cov = np.linalg.inv(posterior_precision)
        posterior_mean = np.dot(posterior_cov, 
                                np.dot(tau_sigma_inv, self.pi) + np.dot(np.dot(P.T, omega_inv), Q))
        
        return posterior_mean
    
    def posterior_covariance(self, P, omega=None):
        """
        Calculate posterior covariance matrix.
        
        M = [(tau*Sigma)^-1 + P'*Omega^-1*P]^-1
        Sigma_posterior = Sigma + M
        
        Parameters:
        -----------
        P : array-like
            Pick matrix
        omega : array-like, optional
            View uncertainty matrix
        
        Returns:
        --------
        array : Posterior covariance matrix
        """
        P = np.array(P)
        
        if omega is None:
            omega = self.tau * np.dot(np.dot(P, self.sigma), P.T)
            if omega.ndim == 0:
                omega = np.array([[omega]])
            elif omega.ndim == 1:
                omega = np.diag(omega)
        
        omega = np.atleast_2d(omega)
        
        tau_sigma_inv = np.linalg.inv(self.tau * self.sigma)
        omega_inv = np.linalg.inv(omega)
        
        M = np.linalg.inv(tau_sigma_inv + np.dot(np.dot(P.T, omega_inv), P))
        
        return self.sigma + M
    
    def optimal_weights(self, P=None, Q=None, omega=None):
        """
        Calculate optimal weights using mean-variance with Black-Litterman returns.
        
        w* = (delta * Sigma)^-1 * E[R]
        
        Parameters:
        -----------
        P : array-like, optional
            Pick matrix for views
        Q : array-like, optional
            View expected returns
        omega : array-like, optional
            View uncertainty matrix
        
        Returns:
        --------
        dict : Optimal weights and portfolio statistics
        """
        if P is None or Q is None:
            # No views - return market equilibrium weights
            expected_returns = self.pi
        else:
            expected_returns = self.posterior_returns(P, Q, omega)
        
        # Calculate optimal weights
        sigma_inv = np.linalg.inv(self.sigma)
        weights = np.dot(sigma_inv, expected_returns) / self.delta
        
        # Normalize weights to sum to 1
        weights = weights / np.sum(weights)
        
        # Calculate portfolio statistics
        port_return = np.dot(weights, expected_returns) + self.rf
        port_vol = np.sqrt(np.dot(weights.T, np.dot(self.sigma, weights)))
        sharpe = (port_return - self.rf) / port_vol
        
        return {
            'weights': weights,
            'expected_returns': expected_returns + self.rf,
            'return': port_return,
            'volatility': port_vol,
            'sharpe': sharpe
        }

print("Black-Litterman Optimizer class defined!")

### 4.3 Example: Black-Litterman with Investor Views

In [ ]:
# Define market capitalization weights (approximate real-world weights)
market_weights = np.array([0.30, 0.20, 0.10, 0.15, 0.10, 0.05, 0.05, 0.05])
market_weights = market_weights / market_weights.sum()  # Normalize

# Initialize Black-Litterman optimizer
bl = BlackLittermanOptimizer(
    cov_matrix=cov_matrix,
    market_weights=market_weights,
    risk_aversion=2.5,
    tau=0.05,
    risk_free_rate=0.02,
    asset_names=assets
)

# Display implied equilibrium returns (prior)
print("=" * 60)
print("IMPLIED EQUILIBRIUM RETURNS (Prior)")
print("=" * 60)
print("\nThese are the returns the market 'believes' given current weights:")
print()
for asset, pi, mkt_w in zip(assets, bl.pi, market_weights):
    print(f"  {asset:15s}: {pi + 0.02:.2%} (Mkt Weight: {mkt_w:.1%})")

In [ ]:
# Define investor views
# View 1: US Equity will return 10% (absolute view)
# View 2: US Equity will outperform EM Equity by 3% (relative view)
# View 3: Gold will return 5% (absolute view)

# Asset indices: US_Equity=0, Intl_Equity=1, EM_Equity=2, US_Bonds=3, 
#                Corp_Bonds=4, Commodities=5, REITs=6, Gold=7

# Pick matrix P (3 views x 8 assets)
P = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],      # View 1: US Equity absolute
    [1, 0, -1, 0, 0, 0, 0, 0],     # View 2: US Equity vs EM Equity relative
    [0, 0, 0, 0, 0, 0, 0, 1],      # View 3: Gold absolute
])

# View expected returns Q (excess returns)
Q = np.array([0.10 - 0.02, 0.03, 0.05 - 0.02])  # Excess returns (subtract rf)

# Confidence in views (lower = more confident)
# Using diagonal uncertainty matrix
view_confidence = np.array([0.001, 0.002, 0.001])  # Variance of views
omega = np.diag(view_confidence)

print("INVESTOR VIEWS:")
print("-" * 50)
print("View 1: US Equity will return 10% (absolute)")
print("View 2: US Equity will outperform EM Equity by 3%")
print("View 3: Gold will return 5% (absolute)")
print()

# Calculate portfolio with no views (equilibrium)
equil_portfolio = bl.optimal_weights(P=None, Q=None)

# Calculate portfolio with views
bl_portfolio = bl.optimal_weights(P=P, Q=Q, omega=omega)

print("=" * 60)
print("EQUILIBRIUM PORTFOLIO (No Views)")
print("=" * 60)
print(f"Expected Return: {equil_portfolio['return']:.2%}")
print(f"Volatility:      {equil_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio:    {equil_portfolio['sharpe']:.3f}")

print("\n" + "=" * 60)
print("BLACK-LITTERMAN PORTFOLIO (With Views)")
print("=" * 60)
print(f"Expected Return: {bl_portfolio['return']:.2%}")
print(f"Volatility:      {bl_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio:    {bl_portfolio['sharpe']:.3f}")

print("\nWeight Comparison:")
print("-" * 50)
print(f"{'Asset':<15} {'Equilibrium':<12} {'B-L':<12} {'Change':<10}")
print("-" * 50)
for i, asset in enumerate(assets):
    eq_w = equil_portfolio['weights'][i]
    bl_w = bl_portfolio['weights'][i]
    change = bl_w - eq_w
    print(f"{asset:<15} {eq_w:>10.1%} {bl_w:>10.1%} {change:>+10.1%}")

### 4.4 Visualizing Black-Litterman Impact

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Weight comparison
ax1 = axes[0]
x = np.arange(len(assets))
width = 0.35

bars1 = ax1.bar(x - width/2, equil_portfolio['weights'], width, label='Equilibrium', alpha=0.8)
bars2 = ax1.bar(x + width/2, bl_portfolio['weights'], width, label='Black-Litterman', alpha=0.8)

ax1.set_xlabel('Assets', fontsize=11)
ax1.set_ylabel('Weight', fontsize=11)
ax1.set_title('Portfolio Weights: Equilibrium vs Black-Litterman', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(assets, rotation=45, ha='right')
ax1.legend()
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Expected returns comparison
ax2 = axes[1]
prior_returns = bl.pi + bl.rf
posterior_returns = bl_portfolio['expected_returns']

bars3 = ax2.bar(x - width/2, prior_returns, width, label='Prior (Equilibrium)', alpha=0.8)
bars4 = ax2.bar(x + width/2, posterior_returns, width, label='Posterior (B-L)', alpha=0.8)

ax2.set_xlabel('Assets', fontsize=11)
ax2.set_ylabel('Expected Return', fontsize=11)
ax2.set_title('Expected Returns: Prior vs Posterior', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(assets, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.tight_layout()
plt.show()

### 4.5 Advantages of Black-Litterman

| Advantage | Description |
|-----------|-------------|
| **More Stable** | Starting from equilibrium reduces extreme positions |
| **Intuitive Weights** | Results are closer to market weights by default |
| **Incorporates Views** | Allows systematic integration of investment opinions |
| **Uncertainty Quantification** | Views can be assigned confidence levels |
| **Benchmark Aware** | Naturally produces benchmark-relative positions |
| **Reduces Estimation Error** | Shrinks expected returns toward equilibrium |

### Black-Litterman vs Markowitz

| Aspect | Markowitz | Black-Litterman |
|--------|-----------|-----------------|
| Starting Point | Historical returns | Market equilibrium |
| Views | None | Explicitly incorporated |
| Stability | Low (sensitive to inputs) | High (anchored to market) |
| Extreme Weights | Common | Rare |
| Interpretation | Pure optimization | Bayesian updating |

---

## 5. Risk Parity

### 5.1 Theory and Mathematical Foundation

**Risk Parity** (also called **Equal Risk Contribution**) allocates portfolio risk equally across assets, rather than capital.

### Key Insight
Traditional equal-weight portfolios ($1/N$) have unequal **risk contributions**. A portfolio with 50% stocks and 50% bonds is dominated by equity risk.

### Mathematical Formulation

**Risk Contribution of Asset i:**

$$RC_i = w_i \cdot \frac{\partial \sigma_p}{\partial w_i} = w_i \cdot \frac{(\boldsymbol{\Sigma} \mathbf{w})_i}{\sigma_p}$$

Where:
- $RC_i$ = Risk contribution of asset $i$
- $(\boldsymbol{\Sigma} \mathbf{w})_i$ = Marginal contribution to risk
- $\sigma_p = \sqrt{\mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}}$ = Portfolio volatility

**Key Property:** Risk contributions sum to total portfolio risk:
$$\sum_{i=1}^{n} RC_i = \sigma_p$$

### Risk Parity Objective

Equal risk contribution means:
$$RC_i = RC_j \quad \forall i,j$$

Or equivalently:
$$RC_i = \frac{\sigma_p}{n} \quad \forall i$$

### Optimization Problem

$$\min_{\mathbf{w}} \sum_{i=1}^{n} \left( w_i \cdot \frac{(\boldsymbol{\Sigma} \mathbf{w})_i}{\sigma_p} - \frac{\sigma_p}{n} \right)^2$$

Subject to:
- $\mathbf{w}^T \mathbf{1} = 1$
- $w_i \geq 0$